In [18]:
import os
import numpy as np
import tifffile as tiff
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt

# Define input and output directories
test_image_dir = "/mnt/d/lding/CLS/mousumiLiuDinner/set1to5_processed_results/Microtubule_GUV-Liu-20250106T211105Z-001/processed_MT/GUV-MT_obj_tiff_selected_std15"
output_tiff_dir = os.path.join(test_image_dir, "improved_tiff")  # Save improved images in a new folder
output_png_dir = os.path.join(test_image_dir, "improved_png")  # Save improved images in a new folder

# Ensure the output directory exists
os.makedirs(output_tiff_dir, exist_ok=True)
os.makedirs(output_png_dir, exist_ok=True)

# Function to process and improve images
def improve_image(image):
    # Convert to 8-bit if needed
    if image.dtype != np.uint8:
        image = (image * 240)
        image[image>=255]=255
        image = image.astype(np.uint8)
    
    # Apply threshold to segment the object
    _, binary_mask = cv2.threshold(image, 18.8, 255, cv2.THRESH_BINARY)

    # Find contours of the segmented object
    contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if not contours:
        return image  # If no object found, return original

    # Get the outermost object's contour
    largest_contour = max(contours, key=cv2.contourArea)

    # Create a mask for the contour
    contour_mask = np.zeros_like(image, dtype=np.uint8)
    cv2.drawContours(contour_mask, [largest_contour], -1, 255, thickness=2)

    contour_mask = contour_mask * binary_mask
    
    # fig, ax = plt.subplots(1, 3, figsize=(10, 5))
    # ax[0].imshow(image, cmap="gray")
    # ax[0].set_title("Original Image")
    # ax[0].axis("off")

    # ax[1].imshow(contour_mask, cmap="gray")
    # ax[1].set_title("Contour Mask")
    # ax[1].axis("off")
    

    # ax[2].imshow(image*contour_mask, cmap="gray")
    # ax[2].set_title("Contour Intensity")
    # ax[2].axis("off")

    # Get grayscale values of outermost contour pixels
    outermost_pixels = image[contour_mask > 0]    
    # Compute median grayscale value
    if len(outermost_pixels) > 0:
        median_value = int(np.median(outermost_pixels))
        print(median_value)
    else:
        median_value = 0  # If no valid contour, default to black

    # Fill background with median value
    result_image = image.copy()
    result_image[binary_mask == 0] = median_value

    return result_image

# Process all TIFF images in the directory
for filename in os.listdir(test_image_dir):
    if filename.lower().endswith((".tif", ".tiff")):
        file_path = os.path.join(test_image_dir, filename)
        # print(filename)
        
        # Read TIFF image
        image = tiff.imread(file_path)
        
        # If multi-channel, take the first channel
        if image.ndim == 3:
            image = image[0]
        
        # Apply image improvement
        improved_image = improve_image(image)
        
        # Save improved image
        output_tiff_path = os.path.join(output_tiff_dir, filename)
        tiff.imwrite(output_tiff_path, improved_image.astype(np.uint8))
        cv2.imwrite(os.path.join(output_png_dir, f"{filename[:-4]}.png"), improved_image.astype(np.uint8))
        # break

print(f"Image processing complete! Improved images saved in: {output_tiff_path}")


44
39
37
34
35
28
36
44
27
45
35
27
51
33
32
39
42
42
42
36
44
36
32
35
40
37
28
31
34
33
42
47
42
53
53
47
36
47
39
43
47
45
43
46
41
53
48
30
91
80
65
87
86
87
79
72
65
67
86
64
93
96
82
81
58
79
60
84
85
63
82
73
66
55
84
50
59
70
56
56
50
50
95
94
63
84
95
56
63
62
70
37
37
36
50
37
35
33
32
36
33
36
38
36
34
37
37
37
40
39
36
34
33
33
35
35
36
36
34
35
32
46
43
46
51
62
49
50
47
44
47
41
46
47
49
45
48
52
49
47
51
59
49
54
50
45
45
50
42
46
61
52
54
51
56
51
49
49
48
44
52
50
52
47
46
47
52
50
51
48
52
49
46
51
50
55
47
45
45
47
61
47
52
50
46
51
51
51
47
51
49
51
51
50
57
76
73
53
45
54
49
50
55
51
52
48
52
51
49
70
33
37
38
35
30
37
33
36
35
31
43
66
33
25
41
43
38
41
28
43
50
29
61
41
34
28
32
52
59
33
50
48
34
33
39
32
31
45
51
61
25
34
32
31
42
34
26
44
38
30
40
63
49
59
44
92
49
64
54
63
49
39
34
59
106
72
47
31
36
28
41
33
27
29
42
29
28
31
34
33
44
38
33
41
37
36
34
40
48
39
55
42
37
37
47
50
43
35
34
25
29
27
38
34
30
35
35
46
38
40
47
31
38
35
30
41
41
30
50
49
51
49
50
